# A.6 More About Sorting（关于排序）

与python内建的list中的sort相同，ndarray的sort也是直接在原本的数组上进行排序的（in-place sort，原地排序，即不返回新的备份）：

In [1]:
import numpy as np

In [2]:
arr = np.random.randn(6)

In [3]:
arr.sort()

In [4]:
arr

array([-0.946505  , -0.9262575 , -0.90163538, -0.34469782,  0.04871348,
        0.54881372])

对数组原地排序的时候，如果这个数组是一个ndarry的视图（view），原来的数组也会被更改：

In [8]:
arr = np.random.randn(3, 5)
arr

array([[ 1.21383235, -0.43370477, -0.12812113,  2.51898887,  1.07894377],
       [-0.7253269 , -0.14387463,  1.67444032,  1.5689391 , -0.94211142],
       [-0.39496152,  0.29593369,  0.58732919, -1.03388524,  1.67820982]])

In [9]:
arr[:, 0].sort() # sort first column values in-place
arr

array([[-0.7253269 , -0.43370477, -0.12812113,  2.51898887,  1.07894377],
       [-0.39496152, -0.14387463,  1.67444032,  1.5689391 , -0.94211142],
       [ 1.21383235,  0.29593369,  0.58732919, -1.03388524,  1.67820982]])

另一方面，numpy.sort会创建一个新的，排序后的备份：

In [10]:
arr = np.random.randn(5)
arr

array([-0.0732929 , -0.31510255,  1.18799858, -0.08450705,  0.32460573])

In [11]:
np.sort(arr)
arr

array([-0.0732929 , -0.31510255,  1.18799858, -0.08450705,  0.32460573])

所有这些排序方法都可以传入一个轴参数，来单独沿着某个轴来进行排序：

In [12]:
arr = np.random.randn(3, 5)
arr

array([[-0.96555113, -0.43740203,  0.36473482,  0.32591191, -0.27968895],
       [-0.58974087, -0.75956881,  0.07783405,  1.02131178, -0.02992581],
       [-1.34466175, -0.52406953,  2.71523409, -2.33036328, -1.24979854]])

In [14]:
arr.sort(axis=1)
arr

array([[-0.96555113, -0.43740203, -0.27968895,  0.32591191,  0.36473482],
       [-0.75956881, -0.58974087, -0.02992581,  0.07783405,  1.02131178],
       [-2.33036328, -1.34466175, -1.24979854, -0.52406953,  2.71523409]])

我们可能注意到没有一个sort方法提供降序排序，这里要用到一个trick（小技巧），对于python的list对象，`list[::-1]`可以返回倒序的list，对于ndarray也可以这么做：

In [15]:
arr[:, ::-1]

array([[ 0.36473482,  0.32591191, -0.27968895, -0.43740203, -0.96555113],
       [ 1.02131178,  0.07783405, -0.02992581, -0.58974087, -0.75956881],
       [ 2.71523409, -0.52406953, -1.24979854, -1.34466175, -2.33036328]])

# 1 Indirect Sorts: argsort and lexsort（间接排序：argsort和lexsort）

在数据分析中，我们可能需要按照某个键对数据集重新排序。例如，一个包含学生信息的数据集可能需要先按姓排序，再按名排序。这就是间接排序。给定一个键或多个键，我们想要用一个包含索引的数组来对数据排序。这里有两个方法可以使用，argsort和numpy.lexsort。例如：

In [16]:
values = np.array([5, 0, 1, 3, 2])

In [17]:
indexer = values.argsort()
indexer

array([1, 2, 4, 3, 0])

In [18]:
values[indexer]

array([0, 1, 2, 3, 5])

一个更复杂的例子，按照第一行来对一个二维数组进行重排序：

In [19]:
arr = np.random.randn(3, 5)
arr[0] = values
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [-0.92338916, -1.08404638,  0.70216047,  0.32480148,  0.03588127],
       [ 0.60975619,  1.1017462 ,  1.90992688, -1.81753884,  0.6916296 ]])

In [20]:
arr[:, arr[0].argsort()]

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
       [-1.08404638,  0.70216047,  0.03588127,  0.32480148, -0.92338916],
       [ 1.1017462 ,  1.90992688,  0.6916296 , -1.81753884,  0.60975619]])

lexsort与argsort很像，但是它在有多个键的数组上，实现的是间接的字母排序（lexicographical sort）。假设我们想要按first和last name来排序：

In [21]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])

In [22]:
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])

In [23]:
sorter = np.lexsort((first_name, last_name))
sorter

array([1, 2, 3, 0, 4])

In [24]:
zip(last_name[sorter], first_name[sorter])

在第一次使用lexsort的时候，会有点困惑。因为用于排序的键是先从最后传入的数组开始的（start with the last array passed）。所以，尽管写的是`np.lexsort((first_name, last_name))`，但是会先按last_name排序，再按first_name排序。

# 2 Alternative Sort Algorithms（排序算法的替代物）

一个稳定的排序算法会保存每个元素的相对位置。这对于间接排序是非常重要的，尤其是相对顺序有意义的时候：

In [25]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])

In [26]:
key = np.array([2, 2, 1, 1, 1])

In [27]:
indexer = key.argsort(kind='mergesort')
indexer

array([2, 3, 4, 0, 1])

In [28]:
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

唯一稳定的是mergesort算法，能保证`O(nlogn)`的复杂度，但是其变平均效果是低于默认的quicksort算法。下表列出了几种算法：

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/pyogf.png)

# 3 Partially Sorting Arrays（局部排序数组）

排序的一个目的是找出数组中的最大最小值。numpy中有优化的方法，numpy.partition和np.argpartition，可以选出k个最小的元素：

In [31]:
np.random.seed(12345)
np.set_printoptions(precision=4)

In [32]:
arr = np.random.randn(20)
arr

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658,  1.3934,  0.0929,
        0.2817,  0.769 ,  1.2464,  1.0072, -1.2962,  0.275 ,  0.2289,
        1.3529,  0.8864, -2.0016, -0.3718,  1.669 , -0.4386])

In [33]:
np.partition(arr, 3)

array([-2.0016, -1.2962, -0.5557, -0.5194, -0.3718, -0.4386, -0.2047,
        0.2817,  0.769 ,  0.4789,  1.0072,  0.0929,  0.275 ,  0.2289,
        1.3529,  0.8864,  1.3934,  1.9658,  1.669 ,  1.2464])

调用`partition(arr, 3)`的结果中，第一行的三个元素是最小的三个，不过没有特定顺序。np.argpartition，和numpy.argsort相似，返回的索引会把数据重新整理为对应的顺序：

In [34]:
indices = np.argpartition(arr, 3)
indices

array([16, 11,  3,  2, 17, 19,  0,  7,  8,  1, 10,  6, 12, 13, 14, 15,  5,
        4, 18,  9])

In [35]:
arr.take(indices)

array([-2.0016, -1.2962, -0.5557, -0.5194, -0.3718, -0.4386, -0.2047,
        0.2817,  0.769 ,  0.4789,  1.0072,  0.0929,  0.275 ,  0.2289,
        1.3529,  0.8864,  1.3934,  1.9658,  1.669 ,  1.2464])

# 4 numpy.searchsorted: Finding Elements in a Sorted Array（numpy.searchsorted：找到排序后数组中的元素）

searchsorted是一个数组方法，能在一排序后的数组上仅需二分搜索，并返回目标可以插入的位置：

In [36]:
arr = np.array([0, 1, 7, 12, 15])
arr.searchsorted(9)

3

我们也可以传入一个数组，得到对应的索引：

In [37]:
arr.searchsorted([0, 8, 11, 16])

array([0, 3, 3, 5])

对于0元素，返回的同样是0。这是因为默认返回的索引是在一组数字相同的情况下，最左边的位置：

In [38]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])
arr.searchsorted([0, 1])

array([0, 3])

In [39]:
arr.searchsorted([0, 1], side='right')

array([3, 7])

这里介绍searchsorted的另一个用法，假设我们有一个数组从0到10000，还有一个数组表示边界（bucket edges），我们相用边界把这个数组分给多组：

In [40]:
data = np.floor(np.random.uniform(0, 10000, size=50))
data

array([ 9940.,  6768.,  7908.,  1709.,   268.,  8003.,  9037.,   246.,
        4917.,  5262.,  5963.,   519.,  8950.,  7282.,  8183.,  5002.,
        8101.,   959.,  2189.,  2587.,  4681.,  4593.,  7095.,  1780.,
        5314.,  1677.,  7688.,  9281.,  6094.,  1501.,  4896.,  3773.,
        8486.,  9110.,  3838.,  3154.,  5683.,  1878.,  1258.,  6875.,
        7996.,  5735.,  9732.,  6340.,  8884.,  4954.,  3516.,  7142.,
        5039.,  2256.])

In [41]:
bins = np.array([0, 100, 1000, 5000, 10000])

我们用searchsorted来得到每个数组点所在的区间（比如1属于区间[0,100)）:

In [42]:
labels = bins.searchsorted(data)
labels

array([4, 4, 4, 3, 2, 4, 4, 2, 3, 4, 4, 2, 4, 4, 4, 4, 4, 2, 3, 3, 3, 3, 4,
       3, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4, 4, 3,
       3, 4, 4, 3])

然后，与pandas的groupby结合起来：

In [43]:
import pandas as pd

In [44]:
pd.Series(data).groupby(labels).mean()

2     498.000000
3    3064.277778
4    7389.035714
dtype: float64